# Data preparation

In [ ]:
import pandas as pd

In [102]:
corpus = pd.read_csv("./data/corpus.tsv", delimiter="\t", names=["qid","query"])
queries = pd.read_csv("./data/queries.tsv", delimiter="\t", names=["qid","query"])

qrels_test = pd.read_csv("./data/qrels.test.tsv", delimiter="\t", names=["qid","q0","docid","label"]).drop(columns="q0")
qrels_train = pd.read_csv("./data/qrels.train.tsv", delimiter="\t", names=["qid","q0","docid","label"]).drop(columns="q0")

topics_test = pd.read_csv("./data/topics.test.tsv", delimiter="\t", names=["qid","query"])
topics_train = pd.read_csv("./data/topics.train.tsv", delimiter="\t", names=["qid","query"])

In [122]:
topics_train["query"] = topics_train["query"].astype(str)

# Indexing

In [ ]:
import pyterrier as pt

if not pt.started():
    pt.init()

In [ ]:
def trec_generator(filepath, delim, verbose=False):
    file = open(filepath)
    file_reader = csv.reader(file, delimiter=delim)
    
    UP = "\x1B[3A"
    CLR = "\x1B[0K"
    print("\n\n")  # set up blank lines so cursor moves work
    for i, (docno, text) in enumerate(file_reader):
        if i % 100000 == 0 and verbose:
            print(f'{UP}Generated {i}{CLR}\n')
        yield {'docno': docno, 'text': text}


def create_index():
    start = time.time()
    index_path = "./index"
    indexer = pt.IterDictIndexer(index_path, verbose=True, overwrite=True)
    trec_iter = trec_generator("./data/collection.tsv", '\t', verbose=True)
    indexref = indexer.index(trec_iter)
    end = time.time()
    print("Indexing took: ", str(end - start))

# Start indexing (takes 7 minutes)
# create_index()

In [ ]:
index = pt.IndexFactory.of("./index")

# Retrieval

In [2]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.k_1": 0.9, "bm25.b": 0.4}, verbose=True)
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", verbose=True)

pipeline = bm25 >> tfidf
df = pipeline.transform(topics_train_pyterrier)
# Our algorithm did not manage to run beyond this point

NameError: name 'pt' is not defined